In [1]:
import os
import collections
import pymysql
import pymysql.cursors

In [13]:
%%time

# Дополним посты на ПостНауке именами авторов.

conn = pymysql.connect(host="localhost",
                       user="root",
                       password="",
                       db="postnauka",
                       charset="utf8",
                       cursorclass=pymysql.cursors.DictCursor)

authors_names = collections.defaultdict(list)

try:
    with conn.cursor() as cur:
        q = """
        -- Получить имена авторов
        select tr.object_id as post_id, t.term_id as author_id, tt.description as author_name
        from pn_term_taxonomy tt
        join pn_terms t on (t.term_id = tt.term_id)
        join pn_term_relationships tr on (tr.term_taxonomy_id = tt.term_taxonomy_id)
        join pn_posts p on (p.id = tr.object_id)
        where p.post_type = 'post' and p.post_status = 'publish' and tt.taxonomy = 'author'
        order by tr.object_id, t.term_id
        """
        cur.execute(q)
        for row in cur:
            doc_id = row["post_id"]
            author_id = row["author_id"]
            author_str = row["author_name"].split()[:-3]
            author_name = " ".join(author_str[:len(author_str) // 2])
            authors_names[doc_id].append((author_id, author_name))
finally:
    conn.close()

CPU times: user 96 ms, sys: 0 ns, total: 96 ms
Wall time: 1.3 s


In [21]:
%%time

# Дополним существующий датасет именами авторов из MySQL дампа.

for doc_id, authors in authors_names.items():
    with open("../datasets/postnauka/raw_data/meta/%s_meta.txt" % doc_id, "a") as meta_file:
        for author_id, author_name in authors:
            meta_file.write("author_name\t%d\t%s\n" % (author_id, author_name))

CPU times: user 265 ms, sys: 128 ms, total: 393 ms
Wall time: 836 ms


---